In [1]:
# Install library for obtaining time of execution

!pip install ipython-autotime

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# import libraries

%load_ext autotime
import boto3
import os

time: 1.98 s (started: 2023-07-06 06:52:48 +00:00)


In [3]:
# install requiremnts for yolo v5

!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 13.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 48.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 625.9/625.9 kB 15.0 MB/s eta 0:00:0000:01
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 44.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 100.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.4 MB/s eta 0:00:0000:0100:01
  

In [2]:
# Check dataset sizes

In [5]:
len(os.listdir("/home/ec2-user/SageMaker/full_data/train/images/"))

91573

time: 1.58 s (started: 2023-06-16 11:47:48 +00:00)


In [6]:
len(os.listdir("/home/ec2-user/SageMaker/full_data/val/images/"))

7000

time: 7.72 ms (started: 2023-06-16 11:47:50 +00:00)


In [7]:
len(os.listdir("/home/ec2-user/SageMaker/full_data/train/labels/"))

91573

time: 1.48 s (started: 2023-06-16 11:47:52 +00:00)


In [14]:
import pandas as pd
df=pd.read_excel("/home/ec2-user/SageMaker/YOLOv5-Lite/train_data_cleaning_new.xlsx")
df.head()

Image   IOU  Confidence o_class p_class   
0                            00   NaN         NaN   other   other  \
1                        000000   NaN         NaN   other   other   
2                  00000001_000  0.85        0.89       0     cat   
3  00000001_000__hue_saturation  0.85        0.89       0     cat   
4                  00000001_005  0.79        0.92       0     cat   

                  bb_c            pred_bb_c  
0                  NaN                  NaN  
1                  NaN                  NaN  
2  (103, 72, 317, 218)  [118, 69, 321, 227]  
3  (103, 72, 317, 218)  [121, 70, 319, 227]  
4  (-5, -41, 265, 221)     [4, 1, 270, 217]

time: 17.6 s (started: 2023-06-16 08:58:32 +00:00)


In [16]:
df1=df[(df["IOU"]<0.4) & ((df["o_class"]==0) | (df["o_class"]==1))]
ign_imgs=df1["Image"].values
for i in ign_imgs:
    if os.path.exists(f"/home/ec2-user/SageMaker/full_data/train/images/{i}.jpg"):
        os.remove(f"/home/ec2-user/SageMaker/full_data/train/images/{i}.jpg")
        os.remove(f"/home/ec2-user/SageMaker/full_data/train/labels/{i}.txt")

time: 67.3 ms (started: 2023-06-16 08:59:55 +00:00)


In [17]:
s=["gauss","hue","blur"]
c=0
for i in os.listdir("/home/ec2-user/SageMaker/full_data/train/images/"):
    j=i.split(".")[0]
    for k in s:
        if k in j:
            os.remove(f"/home/ec2-user/SageMaker/full_data/train/images/{i}")
            os.remove(f"/home/ec2-user/SageMaker/full_data/train/labels/{j}.txt")
            c=c+1
            break

time: 511 ms (started: 2023-06-16 08:59:57 +00:00)


In [11]:
import pandas as pd
path = "/home/ec2-user/SageMaker/full_data/val/labels/"

def bb_box_capping(path):
    for i in os.listdir(path):
        gt = path + i

        if (os.path.getsize(gt) != 0):
            gt_l = pd.read_csv(gt,header = None,sep=' ') 

            for row in range(0,len(gt_l)):
                box = gt_l.iloc[row].values
                bb_c = [1 if x > 1 else x for x in box[1:]]
                bbc = [0 if x < 0 else x for x in bb_c]
                bbc.insert(0, box[0])
                gt_l.iloc[row] = bbc
            # gt_l.to_csv(path + i, sep='\t', index=False,header=False)
            with open(f"/home/ec2-user/SageMaker/full_data/val/labels/{i}","w") as file:
                file.write(str(bbc[0]))
                file.write(" ")
                file.write(str(bbc[1]))
                file.write(" ")
                file.write(str(bbc[2]))
                file.write(" ")
                file.write(str(bbc[3]))
                file.write(" ")
                file.write(str(bbc[4]))
        else:
            continue
    print("Done")



bb_box_capping(path)

Done
time: 10 s (started: 2023-06-16 12:12:03 +00:00)


In [9]:
if os.path.exists(f"/home/ec2-user/SageMaker/full_data/val/labels/.ipynb_checkpoints"):
    print(1)

time: 1.21 ms (started: 2023-06-16 12:09:58 +00:00)


In [ ]:
!python train.py  --epochs 100 --data train_data.yaml --weights yolov5n.pt --cfg yolov5n.yaml  --batch-size 64 --imgsz 256

train: weights=yolov5n.pt, cfg=yolov5n.yaml, data=train_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 3 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-178-ga199480 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=

In [9]:
## Train for 100 epochs


!python train.py  --epochs 100 --data train_data.yaml --weights yolov5n.pt --cfg yolov5n.yaml  --batch-size 64 --imgsz 256

time: 294 µs (started: 2023-06-16 08:55:29 +00:00)


In [10]:
# Resume training
# since the above one is interuptted , we are resume the above training.


!python train.py --resume runs/train/exp2/weights/last.pt

time: 294 µs (started: 2023-06-16 08:55:42 +00:00)


In [ ]:
## Train for 130 epochs


!python train.py  --epochs 30 --data train_data.yaml --weights runs/train/exp8/weights/best.pt --cfg yolov5n.yaml  --batch-size 64 --imgsz 256

train: weights=runs/train/exp8/weights/best.pt, cfg=yolov5n.yaml, data=train_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=64, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 23 (delta 9), reused 9 (delta 2), pack-reused 0
Unpacking objects: 100% (23/23), 31.69 KiB | 2.88 MiB/s, done.
From https://github.com/ultralytics/yolov5
   3812a1a..2334aa7  master              -> origin/master
   934e881..f91d

In [4]:
# inference on 100 epoch best weights


!python detect.py --weights runs/train/exp9/weights/best.pt --source /home/ec2-user/SageMaker/full_data/val/images/  --data train_data.yaml --img-size 256 --conf-thres 0.4 --iou-thres 0.4 --save-txt --save-conf

detect: weights=['runs/train/exp9/weights/best.pt'], source=/home/ec2-user/SageMaker/full_data/val/images/, data=train_data.yaml, imgsz=[256, 256], conf_thres=0.4, iou_thres=0.4, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-178-ga199480 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
image 1/7000 /home/ec2-user/SageMaker/full_data/val/images/00000001_012.jpg: 192x256 1 cat, 72.4ms
image 2/7000 /home/ec2-user/SageMaker/full_data/val/images/00000001_027.jpg: 192x256 1 cat, 7.9ms
image 3/7000 /home/ec2-user/SageMaker/full_data/val/images/00000001_029.jpg: 224x256 1 cat, 111.0ms
image 4/7000 /

In [4]:
# inference on 100 epoch best weights


!python detect.py --weights runs/train/exp9/weights/best.pt --source Untitled_video_Made_with_Clipchamp.mp4  --data train_data.yaml --img-size 256 --conf-thres 0.4 --iou-thres 0.4 --save-txt --save-conf

detect: weights=['runs/train/exp9/weights/best.pt'], source=Untitled_video_Made_with_Clipchamp.mp4, data=train_data.yaml, imgsz=[256, 256], conf_thres=0.4, iou_thres=0.4, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-178-ga199480 Python-3.10.10 torch-2.0.1+cu117 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
video 1/1 (1/210) /home/ec2-user/SageMaker/yolov5/Untitled_video_Made_with_Clipchamp.mp4: 160x256 (no detections), 71.9ms
video 1/1 (2/210) /home/ec2-user/SageMaker/yolov5/Untitled_video_Made_with_Clipchamp.mp4: 160x256 (no detections), 9.5ms
video 1/1 (3/210) /home/ec2-user/SageMaker/yolov5/Untitled_video_Made_with_C

In [13]:
# Detection on 120 epoch model

 !python detect.py --weights runs/train/exp3/weights/best.pt --source /home/ec2-user/SageMaker/full_data/val/images/  --data train_data.yaml --img-size 256 --conf-thres 0.4 --iou-thres 0.4 --save-txt --save-conf

time: 342 µs (started: 2023-06-16 08:56:12 +00:00)


In [ ]:
# zip -r -X archive_name.zip folder_to_compress